In [2]:
%pip install tslearn

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 773 kB 3.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 56.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.6 MB 50.3 MB/s eta 0:00:01
     |████████████████████████████████| 18.3 MB 55.0 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [4]:
# from keras.preprocessing import sequence
# from keras.models import Sequential
# from keras.layers import Dense, Activation, Embedding, LSTM, Dropout
# from keras.callbacks import EarlyStopping, ModelCheckpoint
# from keras.callbacks import callbacks
import matplotlib.pyplot as plt
from tslearn.metrics import dtw
from tslearn.barycenters import softdtw_barycenter
from tslearn.preprocessing import TimeSeriesScalerMinMax, TimeSeriesScalerMeanVariance, \
    TimeSeriesResampler
from tslearn.clustering import TimeSeriesKMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score, accuracy_score, classification_report, precision_score, confusion_matrix
import numpy as np
from pyspark.sql import *
import pandas as pd

In [5]:
cols = ['eeg c3', 'eeg c4', 'eeg cz', 'eeg f3', 'eeg f4', 'eeg f7', 'eeg f8', 'eeg fp1', 'eeg fp2', 'eeg fz', 'eeg o1', 'eeg o2', 'eeg p3', 'eeg p4', 'eeg pz', 'eeg t3', 'eeg t4', 'eeg t5', 'eeg t6']

In [6]:
# spark = SparkSession.builder.appName('eeg').getOrCreate()
# df=spark.read.csv('/wsu/home/groups/eeg/eeg-data/csv/eeg*.csv', mode="DROPMALFORMED",inferSchema=True, header = True)
df = pd.read_csv("/wsu/home/groups/eeg/eeg-data/train/0.csv")

[      Unnamed: 0    ecg ekg    eeg c3    eeg c4    eeg cz    eeg f3    eeg f4  \
 0              0   0.308676  0.003500  0.003500  0.003500  0.003500  0.003500   
 1              1   5.191484  0.003500  0.003500  0.003500  0.003500  0.003500   
 2              2  27.469299  0.613851 -0.149088  0.156088  0.308676 -0.301676   
 3              3  60.123082  1.681965 -0.301676  0.156088  0.613851 -0.759439   
 4              4  68.667997  2.292317 -0.454263  0.308676  0.766439 -1.217202   
 ...          ...        ...       ...       ...       ...       ...       ...   
 6988        6988 -40.279672  0.308676  3.360431 -0.301676 -1.980141  0.156088   
 6989        6989 -39.821908  0.613851  2.902668  0.003500 -2.132729 -0.606851   
 6990        6990 -40.432259  0.766439  1.376790  0.003500 -2.590492 -1.522378   
 6991        6991 -43.636603  0.919027  0.919027 -0.149088 -2.743080 -1.217202   
 6992        6992 -47.603885  1.376790  1.987141 -0.301676 -1.980141  0.156088   
 
         eeg f

In [9]:
def splitDataFrameIntoSmaller(df, chunkSize = 256): 
    listOfDf = list()
    numberChunks = len(df) // chunkSize + 1
    for i in range(numberChunks):
        listOfDf.append(df[i*chunkSize:(i+1)*chunkSize])
    return listOfDf

In [13]:
# # #standardize the data
def standardize(X):
    X_t = []
    for x in X:
        x_t = []
        for row in x.T:
            xt = TimeSeriesScalerMeanVariance().fit_transform(row)
            x_t.append(xt.reshape(1,-1)[0])
        X_t.append(np.array(x_t).T)
    X_t = np.array(X_t)
    return(X_t)

In [14]:
# Make time series shorter
def resample(X, N):
    X_t = []
    for x in X:
        x_t = []
        for row in x.T:
            xt = TimeSeriesResampler(sz=N).fit_transform(row)
            x_t.append(xt.reshape(1,-1)[0])
        X_t.append(np.array(x_t).T)
    X_t = np.array(X_t)
    return X_t

In [15]:
def extract_features(df):
    #load the data
    X, y = df[cols].values, df['label'].values[0]
    return(X, y)

In [ ]:
df_arr= splitDataFrameIntoSmaller(df, chunkSize=256)

In [ ]:
for df in df_arr:
    X,y = extract_features(df)
    X_scaled = TimeSeriesScalerMinMax().fit_transform(X)
    X_std = standardize(X_scaled)
    
    
# #split into training and testing
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.5, random_state=42)

In [ ]:
#normalize the data
X_train = TimeSeriesScalerMinMax().fit_transform(X_train)
X_test = TimeSeriesScalerMinMax().fit_transform(X_test)

In [ ]:
plt.plot(X_train[0].T[0])
plt.plot(X_train[0].T[1])
plt.plot(X_train[0].T[2])
plt.plot(X_train[0].T[3])
plt.show()

In [4]:
# # #standardize the data
def standardize(X):
    X_t = []
    for x in X:
        x_t = []
        for row in x.T:
            xt = TimeSeriesScalerMeanVariance().fit_transform(row)
            x_t.append(xt.reshape(1,-1)[0])
        X_t.append(np.array(x_t).T)
    X_t = np.array(X_t)
    return(X_t)

In [ ]:
X_train = standardize(X_train)
X_test = standardize(X_test)

In [8]:
# plt.plot(X_train[0].T[0])
# plt.plot(X_train[0].T[1])
# plt.plot(X_train[0].T[2])
# plt.plot(X_train[0].T[3])
# plt.show()

In [9]:
# Make time series shorter
def resample(X, N):
    X_t = []
    for x in X:
        x_t = []
        for row in x.T:
            xt = TimeSeriesResampler(sz=N).fit_transform(row)
            x_t.append(xt.reshape(1,-1)[0])
        X_t.append(np.array(x_t).T)
    X_t = np.array(X_t)
    return X_t

In [11]:
# N = 25
# X_train = resample(X_train, N)
# X_test = resample(X_test, N)

In [12]:
# plt.plot(X_train[0].T[0])
# plt.plot(X_train[0].T[1])
# plt.plot(X_train[0].T[2])
# plt.plot(X_train[0].T[3])
# plt.show()

In [13]:
##may or may not be needed
# X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [17]:
callback_arr = [callbacks.EarlyStopping(monitor='binary_accuracy',
                                        verbose=1,
                                        mode='min')]
model = Sequential()
model.add(LSTM(100, return_sequences = True, input_shape = (N, X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(200, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(25))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])

NameError: name 'callbacks' is not defined

In [15]:
model.fit(X_train_res, y_train, validation_data=(X_test_res, y_test), epochs=100, batch_size=256)

NameError: name 'model' is not defined

In [16]:
def print_evaluation(y_sample, labels):
    print(str("Accuracy: {}").format(accuracy_score(y_sample, labels)))
    print(str("Precision: {}").format(precision_score(y_sample, labels)))
    print(str("Recall: {}").format(recall_score(y_sample, labels)))
    print(str("F1-Score: {}").format(f1_score(y_sample, labels)))
    print(confusion_matrix(y_sample, labels))
    print("\n")

In [17]:
#evaluate training
print("Training Results:")
y_pred = model.predict(X_train_res)
pred_label=[]
for elm in y_pred:
    if elm[0]>0.5:
        pred_label.append(1)
    else:
        pred_label.append(0)
print_evaluation(pred_label, y_train)

NameError: name 'model' is not defined

In [18]:
#evalulate testing
print("Testing Results:")
y_pred = model.predict(X_test_res)
pred_label=[]
for elm in y_pred:
    if elm[0]>0.5:
        pred_label.append(1)
    else:
        pred_label.append(0)

print_evaluation(pred_label, y_test)

NameError: name 'model' is not defined

In [19]:
# serialize model to JSON
model_json = model.to_json()
with open(cwd+"\\model\\model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(cwd+"\\model\\model.h5")
print("Saved model to disk")

NameError: name 'model' is not defined